In [1]:
import torch
import torch.nn.utils.prune as prune
from ultralytics import YOLO

In [25]:
def prune_model(model,amount=0.1):
    for module in model.modules():     
        if isinstance(module, torch.nn.Conv2d):
            print(module)
            prune.l1_unstructured(module, name="weight" ,amount=amount)
            prune.remove(module, "weight")
    return model

In [3]:
model=YOLO("../cocoa_diseases_yolo11x_dense/train6/weights/best.pt")
torch_model=model.model
print(torch_model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(96, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C3k2(
      (cv1): Conv(
        (conv): Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(192, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(384, 384, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(384, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
   

In [4]:
val_dense = model.val(
    data="../datasets/cocoa_diseases/cocoa_dataset.yaml",
    imgsz=640,
    batch=2,  # Small batch size
    device=[0,1]  # GPU
)

Ultralytics 8.3.58 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A16, 15000MiB)
                                                       CUDA:1 (NVIDIA A16, 15000MiB)
YOLO11x summary (fused): 464 layers, 56,830,489 parameters, 0 gradients, 194.4 GFLOPs


val: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:04<00:00,  6.45it/s]


                   all         62        289      0.735      0.648       0.73      0.506
          phytophthora         30         39      0.781      0.564      0.678        0.5
               monilia         23         30      0.653      0.767      0.752       0.56
               healthy         52        220      0.771      0.613       0.76      0.457
Speed: 0.9ms preprocess, 67.0ms inference, 0.0ms loss, 1.5ms postprocess per image
Results saved to runs/detect/val


In [26]:
pruned_torch_model=prune_model(torch_model, amount=0.1)
print("Model pruned.")

Conv2d(3, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
Conv2d(96, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
Conv2d(192, 192, kernel_size=(1, 1), stride=(1, 1))
Conv2d(384, 384, kernel_size=(1, 1), stride=(1, 1))
Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
Conv2d(96, 48, kernel_size=(1, 1), stride=(1, 1))
Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1))
Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
Co

In [6]:
model.model=pruned_torch_model
model.save('yolo11x_trained_pruned.pt')
print("Model saved.")

Model saved.


In [7]:
model=YOLO('yolo11x_trained_pruned.pt')

In [8]:
val_sparse= model.val(
    data="../datasets/cocoa_diseases/cocoa_dataset.yaml",
    imgsz=640,
    batch=2,  # Small batch size
    device=[0,1]  # GPU
)

Ultralytics 8.3.58 🚀 Python-3.8.10 torch-1.13.1+cu116 CUDA:0 (NVIDIA A16, 15000MiB)
                                                       CUDA:1 (NVIDIA A16, 15000MiB)


val: Scanning /home/jovyan/ML2/datasets/cocoa_diseases/labels/val.cache... 62 images, 0 backgrounds, 0 corrupt: 100%|██████████| 62/62 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 31/31 [00:04<00:00,  6.43it/s]


                   all         62        289      0.641       0.62      0.629      0.416
          phytophthora         30         39      0.799      0.564      0.644      0.441
               monilia         23         30      0.614      0.733      0.714       0.51
               healthy         52        220       0.51      0.563      0.528      0.296
Speed: 0.9ms preprocess, 66.9ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/val2
